# reading data from files

in this exercise we will read data from files on disk. 

there are various data formats that we may expect to come across. the methods for reading each of these differs slightly. however there are some basic commonalities that i hope to make clear.

# this week's exercise

the `datafiles` folder contains an excel file `example_filters.xlsx` that peter sent me. 

it is an example of the output describing queries from the brandwatch app that peter needs to deals with, and he wants to build a function which can read such files and convert into a dictionary or a list of dictionaries so that he can pass the values into the `get_chart()` function.

the file does *not* contain a flat table, but rather contains two columns, one containing keys, and the other containing values. each query generates a set of rows like that, but not all queries have all the available variables populated so the number of rows varies.

here is an example of the `get_chart` function call:
```
MyChart2 = queries.get_chart(
    name="[CBG] Scotch-Brite: Spring Cleaning", 
    startDate="2017-09-01", 
    endDate="2017-12-01",
    y_axis="volume", 
    x_axis="days", 
    breakdown_by="queries",
    )
```
and the data comes in a spreadsheet format akin to this ![screenshot of example_filters.xlsx](images/excel_sheet_screenshot.png). an acceptable solution first converts this file to csv (using, e.g. `csvkit`).

# the `open()` function

we start with a small and simple csv file listing the rodent inspections of new york city `NY_rodent_inspections_sample_small.csv`, because why not. 

we need to start by opening a file connection to the file with the built-in `open()` command. 

In [45]:
filename_csv = 'datafiles/NY_rodent_inspections_sample_small.csv'
infile = open(filename_csv) 
# what is this object like?
print(type(infile)) # the file handle is an io wrapper and contains metadata about the file connection
print('the file name is', infile.name)        
print('the connection mode is', infile.mode)   # default mode
print('the file encoding is', infile.encoding) # default encoding
#print('the file newline', infile.newlines)     # default newline setting
print('is the file connection closed?', infile.closed) # nope! we just opened it!
#print('does the file connection use line_buffering', infile.line_buffering) # nope, by default.

<class '_io.TextIOWrapper'>
the file name is datafiles/NY_rodent_inspections_sample_small.csv
the connection mode is r
the file encoding is UTF-8
is the file connection closed? False


## the modes of `open()`
- 'r' means open for reading (default)
- 'w' means open for writing, truncating (deleting) the file first
- 'x' means open for exclusive creation, failing if the file already exists
- 'a' means open for writing, appending to the end of the file in case it exists
- 'b' means binary mode
- 't' means text mode (default)
- '+' means open a disk file for updating (reading and writing)

In [2]:
# now we can read the whole file (only stops at eof)
print(infile.read())

INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
BAIT,1,PO12965,3,1011470035,1,01147,0035,104,WEST 76 STREET,10023,990505,223527,40.7802039792471,-73.9774144709456,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"
BAIT,2,PO12966,3,1011470034,1,01147,0034,102,WEST 76 STREET,10023,990516,223521,40.7801875030438,-73.977374757787,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"
BAIT,30,PO16966,3,2043370027,2,04337,0027,620,THWAITES PLACE,10467,1020110,252216,40.8588765781972,-73.8703636422023,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"
BAIT,31,PO13665,3,2037670077,2,03767,0077,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.8313209626148,-73.861994089899,Br

In [49]:
# we can rewind read head to start of file
infile.seek(0)
# if needed, we could also just read a fixed number of characters ahead
print(infile.read(200)) #

INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DAT


In [4]:
print(infile.read(100)) #

E,LOCATION
BAIT,1,PO12965,3,1011470035,1,01147,0035,104,WEST 76 STREET,10023,990505,223527,40.780203


In [50]:
# need to remember to close the file when we are done
infile.close()
print('is the file connection closed?', infile.closed) # it had better be!

is the file connection closed? True


## the `readline()` function: 
- reads/fetches one line.
- returns a single string 
- if size is specified, at most size characters will be read.
- moves the read head forward to the next line
- if the line being read is blank (but not a the last line), `readline()` returns '\n',
- `infile.readline()` returns an empty string if and only if the end of the file has been reached.

In [6]:
infile = open(filename_csv) 
print(infile.readline())  # first line, a header
print(infile.readline())  # second line 
print(infile.readline(50)) # first 50 characters of third line

INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION

BAIT,1,PO12965,3,1011470035,1,01147,0035,104,WEST 76 STREET,10023,990505,223527,40.7802039792471,-73.9774144709456,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"

BAIT,2,PO12966,3,1011470034,1,01147,0034,102,WEST 


In [7]:
# need to remember to close the file when we are done
infile.close()

In [8]:
# the file handler can be iterated over
infile = open(filename_csv, mode='r') # explicitly setting the mode.
for line in infile:
    print('>', line)
infile.close()
print('is the file connection closed?', infile.closed)

> INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION

> BAIT,1,PO12965,3,1011470035,1,01147,0035,104,WEST 76 STREET,10023,990505,223527,40.7802039792471,-73.9774144709456,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"

> BAIT,2,PO12966,3,1011470034,1,01147,0034,102,WEST 76 STREET,10023,990516,223521,40.7801875030438,-73.977374757787,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"

> BAIT,30,PO16966,3,2043370027,2,04337,0027,620,THWAITES PLACE,10467,1020110,252216,40.8588765781972,-73.8703636422023,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"

> BAIT,31,PO13665,3,2037670077,2,03767,0077,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.8313209626148,-73.8

In [9]:
# here is a common way to iterate over the lines:
with open(filename_csv, mode='r') as infile:
    # the enumerate function returns an iterable
    for i, line in enumerate(infile):
        # process each row?
        print('row number:', i, ':', line)
# this way, the file connection is automatically closed when we exit the loop!
print('is the file connection closed?', infile.closed)

row number: 0 : INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION

row number: 1 : BAIT,1,PO12965,3,1011470035,1,01147,0035,104,WEST 76 STREET,10023,990505,223527,40.7802039792471,-73.9774144709456,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"

row number: 2 : BAIT,2,PO12966,3,1011470034,1,01147,0034,102,WEST 76 STREET,10023,990516,223521,40.7801875030438,-73.977374757787,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"

row number: 3 : BAIT,30,PO16966,3,2043370027,2,04337,0027,620,THWAITES PLACE,10467,1020110,252216,40.8588765781972,-73.8703636422023,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"

row number: 4 : BAIT,31,PO13665,3,2037670077,2,03767,

now you can start to see how we **could** use this to extract data from the file

In [10]:
with open(filename_csv, mode='r') as infile:
    headerline_str = infile.readline()  # first line, a header
    header = headerline_str.split(',')
    data_table = []
    for line in infile:
        values = line.split(',')
        row_dict = dict(zip(header, values))
        data_table.append(row_dict)
print(data_table)
print('is the file connection closed?', infile.closed)

[{'INSPECTION_TYPE': 'BAIT', 'JOB_TICKET_OR_WORK_ORDER_ID': '1', 'JOB_ID': 'PO12965', 'JOB_PROGRESS': '3', 'BBL': '1011470035', 'BORO_CODE': '1', 'BLOCK': '01147', 'LOT': '0035', 'HOUSE_NUMBER': '104', 'STREET_NAME': 'WEST 76 STREET', 'ZIP_CODE': '10023', 'X_COORD': '990505', 'Y_COORD': '223527', 'LATITUDE': '40.7802039792471', 'LONGITUDE': '-73.9774144709456', 'BOROUGH': 'Manhattan', 'INSPECTION_DATE': '10/14/2009 12:00:27 PM', 'RESULT': 'Bait applied', 'APPROVED_DATE': '10/14/2009 03:01:46 PM', 'LOCATION\n': '"(40.7802039792471'}, {'INSPECTION_TYPE': 'BAIT', 'JOB_TICKET_OR_WORK_ORDER_ID': '2', 'JOB_ID': 'PO12966', 'JOB_PROGRESS': '3', 'BBL': '1011470034', 'BORO_CODE': '1', 'BLOCK': '01147', 'LOT': '0034', 'HOUSE_NUMBER': '102', 'STREET_NAME': 'WEST 76 STREET', 'ZIP_CODE': '10023', 'X_COORD': '990516', 'Y_COORD': '223521', 'LATITUDE': '40.7801875030438', 'LONGITUDE': '-73.977374757787', 'BOROUGH': 'Manhattan', 'INSPECTION_DATE': '10/14/2009 12:51:21 PM', 'RESULT': 'Bait applied', 'APP

In [11]:
# now we could access individual elements from the data file:
[arow['ZIP_CODE'] for arow in data_table]

['10023', '10023', '10467', '10472', '10024']

don't worry. we won't be doing much of that. as we will see, there are better ways. but let's continue exploring the basic functions. by the way, did you notice a big flaw with the above data table? what if you look consider this:

In [12]:
with open(filename_csv, 'r') as infile:
    headerline_str = infile.readline()  # first line, a header
    header = headerline_str.split(',')
    print('header has', len(header), 'elements, ending with', header[-1])
    for line in infile:
        values = line.split(',')
        print('data line has', len(values), 'elements, ending with', values[-1])

header has 20 elements, ending with LOCATION

data line has 21 elements, ending with  -73.9774144709456)"

data line has 21 elements, ending with  -73.977374757787)"

data line has 21 elements, ending with  -73.8703636422023)"

data line has 21 elements, ending with  -73.861994089899)"

data line has 21 elements, ending with  -73.9805333640688)"



files labelled as csv files can differ by a lot. especially when they contain fields that are free-texts (like mentions do!). csv files you will find in the wild can contain various idiosyncracies:
- they may be tab-delimited, pipe delimited, semi-colon-delimited,...
- they may come from foreign systems that use a different from default end of line pattern ('\r\n')
- they may include quoted fields that contain the delimeter a,b,"bla, bla, bla", 'how many fields in this line?'
- they may contain fields containing end-of-line characters.
- they may contain various control characters that are 'escaped'.

processing a csv file like this is possible, and even easy. but it is laborious. so, there is a module for that.

# the `csv` module

In [13]:
import csv

In [14]:
# simple usage:
with open(filename_csv, newline='') as infile:
    # we first use the csv.reader() function to create a reader object
    rowreader = csv.reader(infile)
    # we can iterate over the rowreader
    for row in rowreader:
        # row is already a list of data values!
        num_vals = len(row)
        print('row number', rowreader.line_num, 'has', num_vals, 'fields and ends on', row[num_vals-1])

row number 1 has 20 fields and ends on LOCATION
row number 2 has 20 fields and ends on (40.7802039792471, -73.9774144709456)
row number 3 has 20 fields and ends on (40.7801875030438, -73.977374757787)
row number 4 has 20 fields and ends on (40.8588765781972, -73.8703636422023)
row number 5 has 20 fields and ends on (40.8313209626148, -73.861994089899)
row number 6 has 20 fields and ends on (40.7830590725833, -73.9805333640688)


In [15]:
# the csv module comes with distinct dialects for parsing csv files:
print(csv.list_dialects())
# we can specify which dialect we want to use:
with open(filename_csv, newline='') as infile:
    # we first use the csv.reader() function to create a reader object
    rowreader = csv.reader(infile, dialect='unix')
    # we can iterate over the rowreader
    for row in rowreader:
        # row is already a list of data values!
        num_vals = len(row)
        print('row number', rowreader.line_num, 'has', num_vals, 'fields and ends on', row[num_vals-1])

['excel', 'excel-tab', 'unix']
row number 1 has 20 fields and ends on LOCATION
row number 2 has 20 fields and ends on (40.7802039792471, -73.9774144709456)
row number 3 has 20 fields and ends on (40.7801875030438, -73.977374757787)
row number 4 has 20 fields and ends on (40.8588765781972, -73.8703636422023)
row number 5 has 20 fields and ends on (40.8313209626148, -73.861994089899)
row number 6 has 20 fields and ends on (40.7830590725833, -73.9805333640688)


if our csv file has a different structure than any of the three given dialects, we can create our own. the `csv` module contains a class called `Sniffer` for helping with that by inspecting our csv file for the structure. 

In [16]:
with open(filename_csv, newline='') as infile:
    # use the regular read() to fetch a sample of the file
    sample = infile.read(2048) 
    inspector = csv.Sniffer() # generate an inspection object
    # does there seem to be a header?
    is_there_a_header = inspector.has_header(sample) 
    # reveal the dialect discovered:
    if is_there_a_header:
        print('file', filename_csv, 'has a header.')
    else:
        print('file', filename_csv, 'has no header!')


with open(filename_csv, newline='') as infile:
    sample = infile.read(2048) 
    inspector = csv.Sniffer() # generate an inspection object
    # guess the dialect of the csv file: delimiter, quote characters, etc
    dialect = inspector.sniff(sample)

    print('the csv file', filename_csv, 'has "' + dialect.delimiter + '" as its delimiter character')
    if dialect.escapechar is not None:
        print('the csv file', filename_csv, 'has "'+str(dialect.escapechar)+'as its escapechar char')
    else:
        print('the csv file', filename_csv, 'has None escapechar character')
    if dialect.doublequote:
        # controls how instances of quotechar appearing inside a field should themselves be quoted.
        # when True, the character is doubled. when False, the escapechar is used as a prefix to 
        # the quotechar. defaults to True.
        print('if a quotechar occurs inside a field, it gets doubled')
    else:
        print('if a quotechar occurs inside a field, it gets escaped by', str(dialect.escapechar))
    print('the csv file', filename_csv, 'has', dialect.quotechar, 'as its', 'quoting char')
    if dialect.lineterminator is None:
        print('the csv file', filename_csv, 'has lineterminator: None')
    elif dialect.lineterminator == '\r':
        print('the csv file', filename_csv, 'has lineterminator: slash-r')
    elif dialect.lineterminator == '\n':
        print('the csv file', filename_csv, 'has lineterminator: slash-n')
    elif dialect.lineterminator == '\r\n':
        print('the csv file', filename_csv, 'has lineterminator: slash-r-slash-n')
    elif dialect.lineterminator == '':
        print('the csv file', filename_csv, 'has lineterminator: blank')
    else:
        print('the csv file', filename_csv, 'has lineterminator: '+ str(dialect.lineterminator))
    
    if dialect.quoting is not None:
        # dialect.quoting controls when quotes should be generated by the writer and recognised by the reader. 
        # It can take on any of the QUOTE_* constants and defaults to QUOTE_MINIMAL.
        #  [QUOTE_ALL, QUOTE_MINIMAL, QUOTE_NONNUMERIC, QUOTE_NONE]
        print('the csv file', filename_csv, 'has quote level', str(dialect.quoting))
    if dialect.skipinitialspace:
        print('the csv file', filename_csv, 'ignores whitespace characters following the delimiter')
    else:
        print('the csv file', filename_csv, 'respects whitespace characters following the delimiter')

file datafiles/NY_rodent_inspections_sample_small.csv has a header.
the csv file datafiles/NY_rodent_inspections_sample_small.csv has "," as its delimiter character
the csv file datafiles/NY_rodent_inspections_sample_small.csv has None escapechar character
if a quotechar occurs inside a field, it gets escaped by None
the csv file datafiles/NY_rodent_inspections_sample_small.csv has " as its quoting char
the csv file datafiles/NY_rodent_inspections_sample_small.csv has lineterminator: slash-r-slash-n
the csv file datafiles/NY_rodent_inspections_sample_small.csv has quote level 0
the csv file datafiles/NY_rodent_inspections_sample_small.csv respects whitespace characters following the delimiter


In [17]:
def report_on_dialect(filename, dialect):
    # report on the dialect discovered:
    # print(dialect)
    print('the csv file', filename, 'has "' + dialect.delimiter + '" as its delimiter charcter')
    if dialect.escapechar is not None:
        print('the csv file', filename, 'has "'+str(dialect.escapechar)+'as its escapechar char')
    else: 
        print('the csv file', filename, 'has None escapechar character')
    if dialect.doublequote:
        # Controls how instances of quotechar appearing inside a field should themselves be quoted.
        # When True, the character is doubled. 
        # When False, the escapechar is used as a prefix to the quotechar. It defaults to True.
        print('if a quotechar occurs inside a field, it gets doubled')
    else:
        print('if a quotechar occurs inside a field, it gets escaped by '+ str(dialect.escapechar))
    
    print('the csv file', filename_csv, 'has', dialect.quoting,           'as its', 'quoting char')
    
    if dialect.lineterminator is None:
        print('the csv file', filename, 'has lineterminator: None')
    elif dialect.lineterminator == '\r':
        print('the csv file', filename, 'has lineterminator: slash-r')
    elif dialect.lineterminator == '\n':
        print('the csv file', filename, 'has lineterminator: slash-n')
    elif dialect.lineterminator == '\r\n':
        print('the csv file', filename, 'has lineterminator: slash-r-slash-n')
    elif dialect.lineterminator == '':
        print('the csv file', filename, 'has lineterminator: blank')
    else:
        print('the csv file', filename, 'has lineterminator: '+ str(dialect.lineterminator))
    
    if dialect.quoting is not None:
        # dialect.quoting controls when quotes should be generated by the writer and recognised by the reader. 
        # It can take on any of the QUOTE_* constants and defaults to QUOTE_MINIMAL.
        #  [QUOTE_ALL, QUOTE_MINIMAL, QUOTE_NONNUMERIC, QUOTE_NONE]
        print('the csv file', filename, 'has quote level', str(dialect.quoting))
    if dialect.skipinitialspace:
        print('the csv file', filename, 'ignores whitespace characters immediately following the delimiter')
    else:
        print('the csv file', filename, 'respects whitespace characters immediately following the delimiter')

def inspect_csv(filename, sample_size=2048):
    with open(filename, newline='') as infile:
        sample = infile.read(sample_size) # fetch the first few rows
        infile.seek(0) # rewind
        sniffer = csv.Sniffer() # inspection class for csv files
        # does there seem to be a header?
        is_there_a_header = sniffer.has_header(sample) 
        if is_there_a_header:
            print('file', filename, 'has a header.')
        else:
            print('file', filename, 'has no header!')

        # guess the delimiter, quote characters
        dialect = sniffer.sniff(sample)
        report_on_dialect(filename, dialect)

inspect_csv('datafiles/iris.csv')

file datafiles/iris.csv has a header.
the csv file datafiles/iris.csv has "," as its delimiter charcter
the csv file datafiles/iris.csv has None escapechar character
if a quotechar occurs inside a field, it gets escaped by None
the csv file datafiles/NY_rodent_inspections_sample_small.csv has 0 as its quoting char
the csv file datafiles/iris.csv has lineterminator: slash-r-slash-n
the csv file datafiles/iris.csv has quote level 0
the csv file datafiles/iris.csv respects whitespace characters immediately following the delimiter


In [18]:
with open(filename_csv) as infile:
    rowreader = csv.reader(infile, delimiter=',')
    for line_num, row in enumerate(rowreader):
        if line_num == 0:
            # first line is a header
            column_header = row
        print(row[0])
    print(f'Processed {line_num} lines.')
    print('header:', column_header)

INSPECTION_TYPE
BAIT
BAIT
BAIT
BAIT
BAIT
Processed 5 lines.
header: ['INSPECTION_TYPE', 'JOB_TICKET_OR_WORK_ORDER_ID', 'JOB_ID', 'JOB_PROGRESS', 'BBL', 'BORO_CODE', 'BLOCK', 'LOT', 'HOUSE_NUMBER', 'STREET_NAME', 'ZIP_CODE', 'X_COORD', 'Y_COORD', 'LATITUDE', 'LONGITUDE', 'BOROUGH', 'INSPECTION_DATE', 'RESULT', 'APPROVED_DATE', 'LOCATION']


## reading csv files to a dict
what the best way to read in a data file can depend on what we want to do with it next. getting the rows as a `list` object may not be the most useful. instead, you may want a dictionary whose keys are the column heads, and whose 

In [19]:
with open(filename_csv, mode='r') as infile:
    # the csv offers another reader function for generating dicts:
    rowreader = csv.DictReader(infile)
    data_table = []
    for rownum, row in enumerate(rowreader):
        # now every row is a dictionary keyed on the values of the first row
        print('row number', rownum, 'has', len(row), 'columns')
        data_table.append(row)
print('read in', len(data_table), 'rows from', filename_csv)

row number 0 has 20 columns
row number 1 has 20 columns
row number 2 has 20 columns
row number 3 has 20 columns
row number 4 has 20 columns
read in 5 rows from datafiles/NY_rodent_inspections_sample_small.csv


In [20]:
# and we can access any column or value directly
[arow['ZIP_CODE'] for arow in data_table]

['10023', '10023', '10467', '10472', '10024']

In [21]:
[arow['LOCATION'] for arow in data_table]

['(40.7802039792471, -73.9774144709456)',
 '(40.7801875030438, -73.977374757787)',
 '(40.8588765781972, -73.8703636422023)',
 '(40.8313209626148, -73.861994089899)',
 '(40.7830590725833, -73.9805333640688)']

## compressed csv files
we often deal with large data files. often these are shipped and stored in a compressed format, most often gnu zipped (.gz). the good news is that we can read zipped files directly and easily by changing only the opener! 

In [22]:
import gzip
filename_zipped='datafiles/NY_rodent_inspections_sample.csv.gz'
with gzip.open(filename_zipped, mode="rt") as infile:
    # eerything else is exactly the same:
    rowreader = csv.DictReader(infile)
    data_table = []
    for rownum, row in enumerate(rowreader):
        data_table.append(row)
print('read in', len(data_table), 'rows from', filename_csv)
print('last 5 locations:')
[arow['LOCATION'] for arow in data_table][-5:]

read in 9999 rows from datafiles/NY_rodent_inspections_sample_small.csv
last 5 locations:


['(40.7077744363728, -73.8017444698633)',
 '(40.8788397683361, -73.9017406808259)',
 '(40.8549066197023, -73.9130302748932)',
 '(40.8293887213765, -73.9121234267685)',
 '(40.8460651323037, -73.9115881298086)']

note that in engineering, statistics, and physical science, it is common to insert comments into csv files. there [currently is no comment overriding mechanism](https://bugs.python.org/issue1225769) built in to the csv module. one could devise a solution using `str.startswith().`

In [23]:
def decomment(csvfile):
    for row in csvfile:
        raw = row.split('#')[0].strip()
        if raw: yield raw

with open(filename_csv) as csvfile:
    reader = csv.reader(decomment(csvfile))
    for row in reader:
        print(row)

['INSPECTION_TYPE', 'JOB_TICKET_OR_WORK_ORDER_ID', 'JOB_ID', 'JOB_PROGRESS', 'BBL', 'BORO_CODE', 'BLOCK', 'LOT', 'HOUSE_NUMBER', 'STREET_NAME', 'ZIP_CODE', 'X_COORD', 'Y_COORD', 'LATITUDE', 'LONGITUDE', 'BOROUGH', 'INSPECTION_DATE', 'RESULT', 'APPROVED_DATE', 'LOCATION']
['BAIT', '1', 'PO12965', '3', '1011470035', '1', '01147', '0035', '104', 'WEST 76 STREET', '10023', '990505', '223527', '40.7802039792471', '-73.9774144709456', 'Manhattan', '10/14/2009 12:00:27 PM', 'Bait applied', '10/14/2009 03:01:46 PM', '(40.7802039792471, -73.9774144709456)']
['BAIT', '2', 'PO12966', '3', '1011470034', '1', '01147', '0034', '102', 'WEST 76 STREET', '10023', '990516', '223521', '40.7801875030438', '-73.977374757787', 'Manhattan', '10/14/2009 12:51:21 PM', 'Bait applied', '10/14/2009 03:02:30 PM', '(40.7801875030438, -73.977374757787)']
['BAIT', '30', 'PO16966', '3', '2043370027', '2', '04337', '0027', '620', 'THWAITES PLACE', '10467', '1020110', '252216', '40.8588765781972', '-73.8703636422023', '

## a recommendation: use csvkit!
finally, for csv files: look into using [csvkit]
(https://csvkit.readthedocs.io/en/1.0.3/tutorial/1_getting_started.html) a collection of utilities for working with csv from the command line, and fixing csv issues.

## the `pandas` module
the pandas module is a generic data science module of python that has a lot of functions. we will talk about it in detail later, but for now just note how easy it makes reading and writing csv files:

In [24]:
import pandas as pd

pandas offers the `read_csv()` function that takes in a name of a tidy csv file and returns a dataframe (a data frame is a table-like structure of columns and rows of tidy data).

In [25]:
rodent_df = pd.read_csv(filename_csv) # yeah. that's it. neat, huh?

`pd.read_csv()` opens, analyzes, reads, parses the csv file and arranges the data into a pd data frame. 

In [26]:
type(rodent_df)

pandas.core.frame.DataFrame

In [46]:
rodent_df.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
0,BAIT,1,PO12965,3,1011470035,1,1147,35,104,WEST 76 STREET,10023,990505,223527,40.780204,-73.977414,Manhattan,10/14/2009 12:00:27 PM,Bait applied,10/14/2009 03:01:46 PM,"(40.7802039792471, -73.9774144709456)"
1,BAIT,2,PO12966,3,1011470034,1,1147,34,102,WEST 76 STREET,10023,990516,223521,40.780188,-73.977375,Manhattan,10/14/2009 12:51:21 PM,Bait applied,10/14/2009 03:02:30 PM,"(40.7801875030438, -73.977374757787)"
2,BAIT,30,PO16966,3,2043370027,2,4337,27,620,THWAITES PLACE,10467,1020110,252216,40.858877,-73.870364,Bronx,11/09/2009 12:59:55 PM,Bait applied,11/10/2009 02:54:52 PM,"(40.8588765781972, -73.8703636422023)"
3,BAIT,31,PO13665,3,2037670077,2,3767,77,1227,WHITEPLAINS ROAD,10472,1022441,242180,40.831321,-73.861994,Bronx,11/09/2009 11:10:16 AM,Bait applied,11/10/2009 02:56:42 PM,"(40.8313209626148, -73.861994089899)"
4,BAIT,38,PO11291,3,1011690057,1,1169,57,2199,BROADWAY,10024,989641,224567,40.783059,-73.980533,Manhattan,11/10/2009 08:40:42 AM,Bait applied,11/17/2009 11:39:11 AM,"(40.7830590725833, -73.9805333640688)"


pandas' `read_csv` does have full flexibility and power when you need it:
```
pandas.read_csv(filepath_or_buffer, sep=', ', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, doublequote=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
```

we will dedicate a later session to manipulating data in a pandas dataframe. (select, filter, group by, join, reshape, etc). but now we move on to json files.

# the `json` module
outside of csv, another very common data format is json. not coincidentally, this hierarchical file format resembles python dicts and lists. given how flexible this format is, the data contained in a json file may not map neatly on to a flat and tidy data table.

the module which will help us read json files is simply called `json`. if only life were always so simple. 

In [27]:
import json
filename_json = 'datafiles/testjson.json' # small sample data in json format

In [28]:
# reading in data is simple. json.load() to read data from files and json.loads() to parse data from a string
# https://docs.python.org/3.7/library/json.html#json.load
# https://docs.python.org/3.7/library/json.html#json.loads
with open(filename_json, 'r') as infile:
    # the load method in the json module 
    data_struct = json.load(infile)
print('read', len(data_struct), 'dataz from', filename_json)

read 5 dataz from datafiles/testjson.json


In [29]:
# json dump() to write to file, and dumps() to write to a string.
# https://docs.python.org/3.7/library/json.html#json.dump
# https://docs.python.org/3.7/library/json.html#json.dumps
print(json.dumps(data_struct[1], sort_keys=False, indent=4))

{
    "text": "Chicago Sun-Times",
    "float": 1.27,
    "datetime": "1948-01-01T14:57:13",
    "boolean": true,
    "time": "14:57:13",
    "date": "1948-01-01",
    "integer": 63
}


In [30]:
print(data_struct[1]['text'])

Chicago Sun-Times


In [31]:
print(json.dumps(dict(zip(['a','b','c'], [10,2,90])), indent=4))
print(json.dumps([1, 'simple', 'list']))

{
    "a": 10,
    "b": 2,
    "c": 90
}
[1, "simple", "list"]


# reading data from excel
like it or loathe it, there are excel files out there, and sometimes we need to extract data from one. luckily, there is a module for that. 

In [13]:
from openpyxl import load_workbook
filename_spreadsheet = 'datafiles/example_filters.xlsx' # this week's example project
spreadsheet = load_workbook(filename_spreadsheet, read_only=True)

In [14]:
# basic iterating over the cells of a spreadsheet
for sheet in spreadsheet:
    print('contents of', sheet.title, 'in', filename_spreadsheet, ":")
    for row in sheet.rows:
        for cell in row:
            print(cell.value)

contents of Sheet1 in datafiles/example_filters.xlsx :
Mention Data Export
None
Mentions
1 to 0 of 0
Project Id
1998247687
Query Id
[1999542744]
Start Date
Sat Jun 30 23:00:00 UTC 2018
End Date
Wed Aug 01 23:00:00 UTC 2018
Search
buy
Author Group
Test author list
Category
Rudeness
Location
[uk]
Page Type
[twitter]
Sentiment
[negative]
Tag
Customer Service
Xcategory
Slowness
Xdomain
www.youtube.com
Xlocation
[ie]
Xpage Type
[news, facebook]


In [17]:
import itertools
import pandas as pd
# but if we happen know we have tidy data (each column is a measure, each row is an obervation) 
# in the excel file, we can use pandas to make it simpler:
filename_spreadsheet = 'datafiles/iris.xlsx' # tidy data set in excel format
spreadsheets = load_workbook(filename_spreadsheet) # simple function call
sheet = spreadsheets.active # top level sheet.
print(type(sheet)) 
print('contents of', sheet.title, 'in', filename_spreadsheet)
data = sheet.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (itertools.islice(r, 1, None) for r in data)
sheet_data_as_df = pd.DataFrame(data, index=idx, columns=cols)

<class 'openpyxl.worksheet.worksheet.Worksheet'>
contents of Sheet 1 - iris in datafiles/iris.xlsx


In [18]:
sheet_data_as_df.head()

,None,None,None,None
sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa


KeyError: 'sepal_length'

# pickled python data
"pickled" files are not human readable. they are compact and efficient binary files, basically a byte stream on disk. one writes them  (the write operation is called a "dump") and they contain exactly the data object as it is stored in the session's memory at the time of dumping. 

we read and write these using the `pickle` module, which works very similar to the `json` module. there are `pickle.load()` (from file) and `pickle.loads()` (from string) functions, as well as `pickle.dump()` (to file) and `pickle.dumps()` (to string).

though efficient, pickle files are not a preferred data format because they cannot be opened or inspected in other programs than python, and may even not be portable from system to system. additionally, they are black boxes and might contain a hidden payload of arbitrary code, so you should be wary to import python pickle files unless you know and trust its origins. 

In [53]:
# write a pickle file:
import pickle
if sheet_data_as_df:
    # write the iris data set to a pickle file.
    filename_pickle = 'datafiles/iris.pickle'
    with open(filename_pickle, mode='wb') as outfile: # note the binary mode! 
        pickle.dump(sheet_data_as_df, outfile)

import os
# let us check whether the file was actually written:
# print(os.listdir('datafiles/'))
print('is the file there?', os.path.isfile(filename_pickle))

is the file there? True


In [43]:
# test reading in the file again
with open(filename_pickle, mode='rb') as infile: # note the binary mode!
    temp = pickle.load(infile)
print(type(temp))
temp.head() # the contents are exactly the same as they were.

<class 'pandas.core.frame.DataFrame'>


# draft of a solution to this week's exercise
reading in peter's problem spreadsheet.

In [1]:
from openpyxl import load_workbook
filename_spreadsheet = 'datafiles/example_filters.xlsx'
#spreadsheet = load_workbook(filename_spreadsheet, read_only=True)
spreadsheet = load_workbook(filename_spreadsheet) # read only files are more limited
sheet = spreadsheet.active
#print(type(sheet))

In [2]:
print(sheet.title)

Sheet1


## accessing rows in sheet

In [88]:
data = sheet.values
# the data object is a generator, meaning we can step through its values with the `next()` method.
# each call to `next()` generates a row in the form of a tuple of columns
first_row = next(data)
sheet_title = first_row[0] # first row, first column (cell A1) contains a title.

file datafiles/example_filters/Example filters - ADAS.xlsx contains the sheet Sheet1 with Chart Data Export in the A1 cell


In [77]:
# collect the key, value pairs in the sheet into a dict
chart_details = {}
for row in data:
    chart_details[row[0]] = row[1]
print(chart_details)

{'Mentions': '1 to 0 of 0', 'Project Id': '1998247687', 'Query Id': '[1999542744]', 'Start Date': 'Sat Jun 30 23:00:00 UTC 2018', 'End Date': 'Wed Aug 01 23:00:00 UTC 2018', 'Search': 'buy', 'Author Group': 'Test author list', 'Category': 'Rudeness', 'Location': '[uk]', 'Page Type': '[twitter]', 'Sentiment': '[negative]', 'Tag': 'Customer Service', 'Xcategory': 'Slowness', 'Xdomain': 'www.youtube.com', 'Xlocation': '[ie]', 'Xpage Type': '[news, facebook]'}


In [79]:
# this only works so far. some files have a lot of rubbish in the footer that needs to be ignored/cleaned
# let's see a problematic file

In [3]:
filename_spreadsheet = 'datafiles/example_filters/Example filters - Nokia.xlsx'
#spreadsheet = load_workbook(filename_spreadsheet, read_only=True)
spreadsheet = load_workbook(filename_spreadsheet) # read only files are more limited
sheet = spreadsheet.active
data = sheet.values
# the data object is a generator, meaning we can step through its values with the `next()` method.
# each call to `next()` generates a row in the form of a tuple of columns
first_row = next(data)
sheet_title = first_row[0] # first row, first column (cell A1) contains a title.
print(
    'file', filename_spreadsheet,
    'contains the sheet', sheet.title, 
    'with', sheet_title, 'in the A1 cell')
# collect the key, value pairs in the sheet into a dict
chart_details = {}
for row in data:
    if row[0] is not None and row[0] is not '':
        chart_details[row[0]] = row[1]
    else: 
        # stop processing data rows on first blank cell in column A:
        break
print(chart_details)

file datafiles/example_filters/Example filters - Nokia.xlsx contains the sheet Sheet1 with Chart Data Export in the A1 cell
{'Dimension 1': 'weeks', 'Dimension 2': 'queries', 'Aggregate': 'volume', 'Start Date': 'Mon Jan 01 00:00:00 UTC 2018', 'End Date': 'Mon Apr 30 23:00:00 UTC 2018', 'Project Id': '1998248269', 'Query Id': '[1999536803]', 'Search': 'nokia', 'Xdomain': 'www.youtube.com', 'Xpage Type': '[facebook]'}


In [9]:
# now just wrap this up as a function
def read_chart_export(filename_spreadsheet):
    # reads chart data export in excel format and extracts the chart data
    # returns a dictionary with the chart properties 
    # assumes:
    #   cell A1 contains a table title
    #   the keys are in cells A2:An, where n is number of properties
    #   the values are in cells B2:Bn 
    #   any rows below a blank cell in the A column are 'rubbish'
    # nb! files are opened as read-write. because read-only files have limited functionality in the openpyxl module
    
    spreadsheet = load_workbook(filename_spreadsheet) 
    sheet = spreadsheet.active
    data = sheet.values
    # the `data` object is a generator, meaning we can step through its values with the `next()` method.
    # each call to `next()` generates a row in the form of a tuple of columns
    first_row = next(data)
    sheet_title = first_row[0] # first row, first column (cell A1) contains a title.
    print('reading exported chart data from sheet "'+sheet_title+'" in', filename_spreadsheet)
    # collect the key, value pairs in the sheet into a dict
    chart_details = {}
    for row in data:
        if row[0] is not None and row[0] is not '':
            chart_details[row[0]] = row[1]
        else: 
            # stop processing data rows on first blank cell in A column:
            break
    return chart_details

In [10]:
# test the function on the example files peter provided:
import os
example_file_dir = 'datafiles/example_filters/'
file_list = os.listdir(example_file_dir)
path_list = [os.path.join(example_file_dir,filename) for filename in file_list]

all_the_chart_dataz = []
for filepath in path_list:
    chart_data = read_chart_export(filepath)
    if chart_data != {}:
        all_the_chart_dataz.append(chart_data)
# a list of the information from all the example files
print(all_the_chart_dataz)

reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Delta.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Rothschild.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - British Museum.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Essity.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Nokia.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Charter.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - ADAS.xlsx
reading exported chart data from sheet "Chart Data Export" in datafiles/example_filters/Example filters - Peel Hunt.xlsx
reading exported chart data from sheet "Char